# Writing a BitGenerator

The standard method to write a bit generator involves writing a Cython `pyx`
file that wraps some C source code containing a high-performance
implementation of a Pseudo RNG. This leads to best-case performance without
creating any external dependencies.

`UserBitGenerator` provides a simple wrapper class that allows users to write
bit generators in pure Python or, if performance is an issue, using Cython or by
accessing functions in a compiled library (e.g., a DLL).

Here we look examine the steps needed to write a pure Python bit
generator and higher-performance generator using Cython.

In [1]:
# Filter out some FutureWarnings

import warnings

warnings.filterwarnings("ignore", "RandomState", FutureWarning)
warnings.filterwarnings("ignore", "Generator", FutureWarning)

## Using Python

The example here beings by writing a class that implements the PCG64
bit generator using the XSL-RR output transformation. While this is not
a complete implementation (it does not support `advance` or `seed`),
it is simple. The key to understanding PCG is that the underlying state
is updated using a [Linear Congruential Generator (LCG)](https://en.wikipedia.org/wik/Linear_congruential_generator)
that uses a 128-bit state, multiplier and increment. The state evolves according to

$$ s_{n+1} = m s_{n} + i $$

where $s$ is the state, $m$ is the multiplier and $i$ is the increment.

The PCG generator then transforms $s_{n+1}$ to produce the the final output. The XLS-RR
output function XORs the upper 64 bits of the state with the lower
64 bits before applying a random rotation of this value.

The code below implements this generator using built-in Python operations
and a little NumPy.

In [2]:
import numpy as np

# The rotation
def rotr_64(value, rot):
    value = np.uint64(value)
    rot = np.uint64(rot)
    return int((value >> rot) | (value << ((-rot) & np.uint(63))))


class PythonPCG64:
    # A 128 bit multiplier
    PCG_DEFAULT_MULTIPLIER = (2549297995355413924 << 64) + 4865540595714422341
    MODULUS = 2 ** 128

    def __init__(self, state, inc):
        """Directly set the state and increment, no seed support"""
        self.state = state
        self.inc = inc
        self._has_uint32 = False
        self._uinteger = 0
        self._next_32 = self._next_64 = None

    def random_raw(self):
        """Generate the next "raw" value, which is 64 bits"""
        state = self.state * self.PCG_DEFAULT_MULTIPLIER + self.inc
        state = state % self.MODULUS
        self.state = state
        return rotr_64((state >> 64) ^ (state & 0xFFFFFFFFFFFFFFFF), state >> 122)

    @property
    def next_64(self):
        """
        Return a callable that accepts a single input. The input is usually
        a void pointer that is cast to a struct that contains the PRNGs
        state. When wiring a bit generator in Python, it is simpler to use
        a closure than to wrap the state in an array, pass it's address as a
        ctypes void pointer, and then to get the pointer in the function.
        """
        def _next_64(void_p):
            return self.random_raw()
        
        self._next_64 = _next_64
        return _next_64

    @property
    def next_32(self):
        """
        Return a callable that accepts a single input. This is identical to
        ``next_64`` except that it return a 32-bit unsigned int. Here we save
        half of the raw 64 bit output for subsequent calls.
        """
        def _next_32(void_p):
            if self._has_uint32:
                self._has_uint32 = False
                return self._uinteger
            next_value = self.random_raw()
            self._has_uint32 = True
            self._uinteger = next_value >> 32
            return next_value & 0xFFFFFFFF

        self._next_32 = _next_32
        return _next_32
    
    @property
    def state_getter(self):
        def f():
            return {"state": self.state, "inc": self.inc}
        return f

    @property
    def state_setter(self):
        def f(value):
            self.state = value["state"]
            self.inc = value["inc"]
        return f


Next, we use `UserBitGenerator` to expose the Python functions to C.
The Python functions are wrapped in Ctypes callbacks under the hood.

In [3]:
from randomgen import PCG64, UserBitGenerator

pcg = PCG64(0, mode="sequence", variant="xsl-rr")
state, inc = pcg.state["state"]["state"], pcg.state["state"]["inc"]
print("Get the state from a seeded PCG64")
print(pcg.state["state"])
prng = PythonPCG64(state, inc)
print("State and increment are identical")
print(prng.state_getter())
python_pcg = UserBitGenerator(prng.next_64, 64, next_32=prng.next_32)
print("First 5 values from PythonPCG64")
print(python_pcg.random_raw(5))
print("Matches official C version")
print(pcg.random_raw(5))

Get the state from a seeded PCG64
{'state': 168573994392035485979667429847573090803, 'inc': 1}
State and increment are identical
{'state': 168573994392035485979667429847573090803, 'inc': 1}
First 5 values from PythonPCG64
[17589501946320304812  4238265634445818190  2962725298318311686
  8662843047148884465 14592389037209137702]
Matches official C version
[17589501946320304812  4238265634445818190  2962725298318311686
  8662843047148884465 14592389037209137702]


`python_pcg` _is_ a bit generator, and so can be used with a NumPy `Generator`.
Here we see the state changes after producing a single standard normal.

In [4]:
from numpy.random import Generator
gen = Generator(python_pcg)
print(f"Before: {prng.state}")
print(f"Std. Normal : {gen.standard_normal()}")
print(f"After: {prng.state}")

Before: 223347365555069667157438957040342502044
Std. Normal : -0.32247746250377424
After: 57149987494923580281468740370759435277


This time the `state_getter` and `state_setter` are used to that the state can be read and set through the bit generator.

In [5]:
python_pcg = UserBitGenerator(prng.next_64, 64, next_32=prng.next_32, state_getter=prng.state_getter, state_setter=prng.state_setter)
python_pcg.state

{'state': 57149987494923580281468740370759435277, 'inc': 1}

Accessing `python_pcg.state` would raise `NotImplementedError`. It is possible to
wire up this function by setting `state_setter` and `state_getter` in `UserBitGenerator`.
These both take callable function

### Performance
We can time `random_raw` to see how fast (**slow**) the pure python version is. It is about 3 orders-of-magnitude (1000x) slower than the C implementation.

In [6]:
%timeit python_pcg.random_raw(1000)
%timeit pcg.random_raw(1000)

2.38 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.03 µs ± 43 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Using numba

A bit generator implemented in Numba can be used through the `UserBitGenerator.from_cfunc` interface. The close block below implements the SplitMix64 generator using numba. This generator mixes the output of an additive congruential generator (LCG with $m=1$) using the [Murmur 3 hash](https://en.wikipedia.org/wiki/MurmurHash). 

The key outputs of the manager class are `next_64`, `next_32`, and `next_double`, which are all decorated using numba's `@cfunc`. 

**Note**: SplitMix64 was used because it is simple to implement. PCG could be implemented with some extra effort needed to update the state since it uses 128-bit unsigned integers which have [no direct support in numba](https://github.com/numba/numba/issues/2600).

In [7]:
import ctypes

import numpy as np
from numba import cfunc, types, carray, jit

from randomgen.generator import Generator
from randomgen.wrapper import UserBitGenerator


murmur_hash_3_sig = types.uint64(types.uint64)


@jit(signature_or_function=murmur_hash_3_sig, inline="always")
def murmur_hash_3(z):
    """The hash function"""
    z = (z ^ (z >> np.uint64(30))) * np.uint64(0xBF58476D1CE4E5B9)
    z = (z ^ (z >> np.uint64(27))) * np.uint64(0x94D049BB133111EB)
    return z ^ (z >> np.uint64(31))


split_mix_next_sig = types.uint64(types.uint64[:])


@jit(signature_or_function=split_mix_next_sig, inline="always")
def splitmix_next(state):
    """Update the state in place and then hash"""
    state[0] += 0x9E3779B97F4A7C15
    return murmur_hash_3(state[0])


class NumbaSplitMix64:
    def __init__(self, state):
        if not isinstance(state, (int, np.integer)) or not (0 <= state < 2 ** 64):
            raise ValueError("state must be a valid uint64")
        # state[0] is the splitmix64 state
        # state[1] contains both the has_uint flag in bit 0
        #   uinteger in bits 32...63
        self._state = np.array([state, 0], dtype=np.uint64)
        self._next_raw = None
        self._next_64 = None
        self._next_32 = None
        self._next_double = None

    @property
    def state_address(self):
        """Get the location in memory of the state NumPy array."""
        return self._state.ctypes.data_as(ctypes.c_void_p)

    @property
    def next_64(self):
        """Same as raw since a 64 bit generator"""

        # Ensure a reference is held
        self._next_64 = self.next_raw

        return self.next_raw

    @property
    def next_32(self):
        """A CFunc generating the next 32 bits"""
        sig = types.uint32(types.CPointer(types.uint64))

        @cfunc(sig)
        def next_32(st):
            # Get the NumPy uint64 array
            bit_gen_state = carray(st, (2,), dtype=np.uint64)
            # We use the first bit to indicate that 32 bits are stored in 32...63
            if bit_gen_state[1] & np.uint64(0x1):
                # Get the upper 32 bits
                out = bit_gen_state[1] >> np.uint64(32)
                # Clear the stored value
                bit_gen_state[1] = 0
                return out
            # If no bits available, genrate a new value
            z = splitmix_next(bit_gen_state)
            # Store the new value always with 1 in bit 0
            bit_gen_state[1] = z | np.uint64(0x1)
            # Return the lower 32 (0...31)
            return z & 0xFFFFFFFF

        # Ensure a reference is held
        self._next_32 = next_32

        return next_32

    @property
    def next_double(self):
        """A CFunc that generates the next ouble"""
        sig = types.double(types.CPointer(types.uint64))

        @cfunc(sig)
        def next_double(st):
            # Get the state
            bit_gen_state = carray(st, (2,), dtype=np.uint64)
            # Return the next value / 2**53
            return (
                np.uint64(splitmix_next(bit_gen_state)) >> np.uint64(11)
            ) / 9007199254740992.0

        # Ensure a reference is held
        self._next_double = next_double

        return next_double

    @property
    def next_raw(self):
        sig = types.uint64(types.CPointer(types.uint64))

        @cfunc(sig)
        def next_64(st):
            # Get the NumPy array containing the state
            bit_gen_state = carray(st, (2,), dtype=np.uint64)
            # Return the next value
            return splitmix_next(bit_gen_state)

        # Ensure a reference is held
        self._next_64 = next_64

        return next_64

    @property
    def state_getter(self):
        """A function that returns the state. This is Python and is not decorated"""
        def f() -> dict:
            return {
                "bit_gen": type(self).__name__,
                "state": self._state[0],
                "has_uint": self._state[1] & np.uint64(0x1),
                "uinteger": self._state[1] >> np.uint64(32),
            }

        return f

    @property
    def state_setter(self):
        """A function that sets the state. This is Python and is not decorated"""
        def f(value: dict):
            name = value.get("bit_gen", None)
            if name != type(self).__name__:
                raise ValueError(f"state must be from a {type(self).__name__}")
            self._state[0] = np.uint64(value["state"])
            temp = np.uint64(value["uinteger"]) << np.uint64(32)
            temp |= np.uint64(value["has_uint"]) & np.uint64(0x1)
            self._state[1] = temp

        return f

We start by instantizing the class and taking a look at the initial state.

In [8]:
# From random.org
state = np.array([0x77, 0x5e, 0xb7, 0x11, 0x14, 0x3f, 0xd1, 0x0e], dtype=np.uint8).view(np.uint64)[0]
nsm = NumbaSplitMix64(state)
nsm.state_getter()

{'bit_gen': 'NumbaSplitMix64',
 'state': 1067703942092643959,
 'has_uint': 0,
 'uinteger': 0}

`from_cfunc` is then used to pass the `CFunc`s, state address pointer and the state getter and setter to `UserBitGenerator`. We see that the state changes after calling `random_raw`.

In [9]:
split_mix = UserBitGenerator.from_cfunc(nsm.next_raw, nsm.next_64, nsm.next_32, nsm.next_double, nsm.state_address, 
                                        state_getter=nsm.state_getter, state_setter=nsm.state_setter)
print(split_mix.state)
print(split_mix.random_raw(2))
print(split_mix.state)

{'bit_gen': 'NumbaSplitMix64', 'state': 1067703942092643959, 'has_uint': 0, 'uinteger': 0}
[6167196305974902360 9866841637329706610]
{'bit_gen': 'NumbaSplitMix64', 'state': 5422389507029489313, 'has_uint': 0, 'uinteger': 0}


Some `Generator` function use 32-bit integers to save bits. `random` with `dtype=np.float32` is one.  After calling this function we see that `has_uint` is now 1.

In [10]:
gen = Generator(split_mix)
print(f"A 32-bit float: {gen.random(dtype=np.float32)}")
print("Notice has_uint is now 1")
gen.state

A 32-bit float: 0.9973556995391846
Notice has_uint is now 1


{'bit_gen': 'NumbaSplitMix64',
 'state': 16823104326352687798,
 'has_uint': 1,
 'uinteger': 1192548665}

### Performance 
We can use `random_raw` function to assess the performance and compare it to PCG64. It is a bit faster as it should be since it is simpler.

In [11]:
%timeit split_mix.random_raw(1000000)

2.81 ms ± 25 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit pcg.random_raw(1000000)

3.07 ms ± 12.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
